In [1]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import os, glob, re
from models.stutterModel import StutterDetectionModel_FC, StutterDetectionModel_LSTM
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from tqdm import tqdm


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
model_fc_name = 'FC_Model'
model_lstm_name = 'LSTM_Model'
epoch = '12'  
# Caminho para o checkpoint
checkpoint_dir = 'checkpoints'
checkpoint_path_fc = os.path.join(checkpoint_dir, f'{model_fc_name}_epoch_{epoch}.pth')
checkpoint_path_lstm = os.path.join(checkpoint_dir, f'{model_lstm_name}_epoch_{epoch}.pth')

# Inicialize o modelo

num_disfluencies = 1  # Saída binária
model_lstm = StutterDetectionModel_LSTM(num_disfluencies)
model_fc = StutterDetectionModel_FC(num_disfluencies)

# Carregue o checkpoint
checkpoint_1 = torch.load(checkpoint_path_fc)
checkpoint_2 = torch.load(checkpoint_path_lstm)
model_fc.load_state_dict(checkpoint_1['model_state_dict'])
model_lstm.load_state_dict(checkpoint_2['model_state_dict'])
model_fc = model_fc.to(device)
model_lstm = model_lstm.to(device)
model_fc.eval() 
model_lstm.eval()

/tmp/ipykernel_3971662/490371944.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_1 = torch.load(checkpoint_path_fc)
/tmp/ipykernel_3971662/490371944.py:17: F

StutterDetectionModel_LSTM(
  (resnet): ResNet18(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [8]:
data_path = '/home/filhoij/Documents/CEIA/disfluency/projeto_residual_bi_lstm/data/marcelo_exemplos/'
mp3_files = glob.glob(os.path.join(data_path, '*.mp3'))
mp3_files

['/home/filhoij/Documents/CEIA/disfluency/projeto_residual_bi_lstm/data/marcelo_exemplos/disf_1_mar.mp3',
 '/home/filhoij/Documents/CEIA/disfluency/projeto_residual_bi_lstm/data/marcelo_exemplos/disf_2_mar.mp3',
 '/home/filhoij/Documents/CEIA/disfluency/projeto_residual_bi_lstm/data/marcelo_exemplos/disf_3_mar.mp3',
 '/home/filhoij/Documents/CEIA/disfluency/projeto_residual_bi_lstm/data/marcelo_exemplos/disf_4_mar.mp3',
 '/home/filhoij/Documents/CEIA/disfluency/projeto_residual_bi_lstm/data/marcelo_exemplos/non_disf_1_mar.mp3',
 '/home/filhoij/Documents/CEIA/disfluency/projeto_residual_bi_lstm/data/marcelo_exemplos/non_disf_2_mar.mp3',
 '/home/filhoij/Documents/CEIA/disfluency/projeto_residual_bi_lstm/data/marcelo_exemplos/Sanduiche-iche - (Ruth Lemos).mp3']

In [32]:
waveform, sample_rate = torchaudio.load(mp3_files[0])
waveform = waveform[:int(sample_rate*3.1)]
mel_spectrogram_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate, 
    n_mels=100,
    n_fft=512,
    win_length=400,
    hop_length=160,
)

mel_spectrogram = mel_spectrogram_transform(waveform)
mel_spectrogram = torch.log(mel_spectrogram + 1e-13)
mel_spectrogram = mel_spectrogram.unsqueeze(0).to(device)

with torch.no_grad():
    output_fc = model_fc(mel_spectrogram)
    output_lstm = model_lstm(mel_spectrogram)
    pred_fc = torch.sigmoid(output_fc).round().item()
    pred_lstm = torch.sigmoid(output_lstm).round().item()

print(f"Predicao {mp3_files[0].split('/')[-1]} FC-MODEL: {pred_fc}")
print(f"Predicao {mp3_files[0].split('/')[-1]} LSTM-MODEL: {pred_lstm}")

Predicao disf_1_mar.mp3 FC-MODEL: 1.0
Predicao disf_1_mar.mp3 LSTM-MODEL: 1.0


/home/filhoij/.local/lib/python3.10/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (100) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(


In [33]:
waveform, sample_rate = torchaudio.load(mp3_files[1])
waveform = waveform[:int(sample_rate*3.1)]
mel_spectrogram_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate, 
    n_mels=100,
    n_fft=512,
    win_length=400,
    hop_length=160,
)

mel_spectrogram = mel_spectrogram_transform(waveform)
mel_spectrogram = torch.log(mel_spectrogram + 1e-13)
mel_spectrogram = mel_spectrogram.unsqueeze(0).to(device)

with torch.no_grad():
    output_fc = model_fc(mel_spectrogram)
    output_lstm = model_lstm(mel_spectrogram)
    pred_fc = torch.sigmoid(output_fc).round().item()
    pred_lstm = torch.sigmoid(output_lstm).round().item()

print(f"Predicao {mp3_files[1].split('/')[-1]} FC-MODEL: {pred_fc}")
print(f"Predicao {mp3_files[1].split('/')[-1]} LSTM-MODEL: {pred_lstm}")

Predicao disf_2_mar.mp3 FC-MODEL: 1.0
Predicao disf_2_mar.mp3 LSTM-MODEL: 1.0


In [34]:
waveform, sample_rate = torchaudio.load(mp3_files[2])
waveform = waveform[:int(sample_rate*3.1)]
mel_spectrogram_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate, 
    n_mels=100,
    n_fft=512,
    win_length=400,
    hop_length=160,
)

mel_spectrogram = mel_spectrogram_transform(waveform)
mel_spectrogram = torch.log(mel_spectrogram + 1e-13)
mel_spectrogram = mel_spectrogram.unsqueeze(0).to(device)

with torch.no_grad():
    output_fc = model_fc(mel_spectrogram)
    output_lstm = model_lstm(mel_spectrogram)
    pred_fc = torch.sigmoid(output_fc).round().item()
    pred_lstm = torch.sigmoid(output_lstm).round().item()


print(f"Predicao {mp3_files[2].split('/')[-1]} FC-MODEL: {pred_fc}")
print(f"Predicao {mp3_files[2].split('/')[-1]} LSTM-MODEL: {pred_lstm}")

Predicao disf_3_mar.mp3 FC-MODEL: 1.0
Predicao disf_3_mar.mp3 LSTM-MODEL: 1.0


In [35]:
waveform, sample_rate = torchaudio.load(mp3_files[3])
waveform = waveform[:int(sample_rate*3.1)]
mel_spectrogram_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate, 
    n_mels=100,
    n_fft=512,
    win_length=400,
    hop_length=160,
)

mel_spectrogram = mel_spectrogram_transform(waveform)
mel_spectrogram = torch.log(mel_spectrogram + 1e-13)
mel_spectrogram = mel_spectrogram.unsqueeze(0).to(device)

with torch.no_grad():
    output_fc = model_fc(mel_spectrogram)
    output_lstm = model_lstm(mel_spectrogram)
    pred_fc = torch.sigmoid(output_fc).round().item()
    pred_lstm = torch.sigmoid(output_lstm).round().item()

print(f"Predicao {mp3_files[3].split('/')[-1]} FC-MODEL: {pred_fc}")
print(f"Predicao {mp3_files[3].split('/')[-1]} LSTM-MODEL: {pred_lstm}")

Predicao disf_4_mar.mp3 FC-MODEL: 1.0
Predicao disf_4_mar.mp3 LSTM-MODEL: 1.0


In [36]:
waveform, sample_rate = torchaudio.load(mp3_files[4])
waveform = waveform[:int(sample_rate*3.1)]
mel_spectrogram_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate, 
    n_mels=100,
    n_fft=512,
    win_length=400,
    hop_length=160,
)

mel_spectrogram = mel_spectrogram_transform(waveform)
mel_spectrogram = torch.log(mel_spectrogram + 1e-13)
mel_spectrogram = mel_spectrogram.unsqueeze(0).to(device)

with torch.no_grad():
    output_fc = model_fc(mel_spectrogram)
    output_lstm = model_lstm(mel_spectrogram)
    pred_fc = torch.sigmoid(output_fc).round().item()
    pred_lstm = torch.sigmoid(output_lstm).round().item()

print(f"Predicao {mp3_files[4].split('/')[-1]} FC-MODEL: {pred_fc}")
print(f"Predicao {mp3_files[4].split('/')[-1]} LSTM-MODEL: {pred_lstm}")

Predicao non_disf_1_mar.mp3 FC-MODEL: 1.0
Predicao non_disf_1_mar.mp3 LSTM-MODEL: 1.0


In [37]:
waveform.shape, sample_rate

(torch.Size([1, 251136]), 48000)

In [27]:
waveform, sample_rate = torchaudio.load(mp3_files[5])
waveform = waveform[:int(sample_rate*3.1)]
mel_spectrogram_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate, 
    n_mels=100,
    n_fft=512,
    win_length=400,
    hop_length=160,
)

mel_spectrogram = mel_spectrogram_transform(waveform)
mel_spectrogram = torch.log(mel_spectrogram + 1e-13)
mel_spectrogram = mel_spectrogram.unsqueeze(0).to(device)

with torch.no_grad():
    output_fc = model_fc(mel_spectrogram)
    output_lstm = model_lstm(mel_spectrogram)
    pred_fc = torch.sigmoid(output_fc).round().item()
    pred_lstm = torch.sigmoid(output_lstm).round().item()

print(f"Predicao {mp3_files[5].split('/')[-1]} FC-MODEL: {pred_fc}")
print(f"Predicao {mp3_files[5].split('/')[-1]} LSTM-MODEL: {pred_lstm}")

Predicao non_disf_2_mar.mp3 FC-MODEL: 1.0
Predicao non_disf_2_mar.mp3 LSTM-MODEL: 1.0


In [40]:
output_fc, sample_rate = torchaudio.load(mp3_files[6])
waveform = waveform[:int(sample_rate*3.1)]
mel_spectrogram_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate, 
    n_mels=100,
    n_fft=512,
    win_length=400,
    hop_length=160,
)

mel_spectrogram = mel_spectrogram_transform(waveform)
mel_spectrogram = torch.log(mel_spectrogram + 1e-13)
mel_spectrogram = mel_spectrogram.unsqueeze(0).to(device)

with torch.no_grad():
    output_fc = model_fc(mel_spectrogram)
    output_lstm = model_lstm(mel_spectrogram)
    pred_fc = torch.sigmoid(output_fc).round().item()
    pred_lstm = torch.sigmoid(output_lstm).round().item()

print(f"Predicao {mp3_files[6].split('/')[-1]} FC-MODEL: {pred_fc}")
print(f"Predicao {mp3_files[6].split('/')[-1]} LSTM-MODEL: {pred_lstm}")

Predicao Sanduiche-iche - (Ruth Lemos).mp3 FC-MODEL: 1.0
Predicao Sanduiche-iche - (Ruth Lemos).mp3 LSTM-MODEL: 1.0
